In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Imports

In [0]:
%matplotlib inline
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, SGD
from keras.losses import mean_squared_error as MSE
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

"""Reader Class for reading data"""
import pickle

class Reader:

    def __init__(self, log_file):
        self._log_file = open(log_file, 'rb')

    def read(self):
        end = False
        observations = []
        actions = []

        while not end:
            try:
                log = pickle.load(self._log_file)
                for entry in log:
                    step = entry['step']
                    observations.append(step[0])
                    actions.append(step[1])
            except EOFError:
                end = True

        return observations, actions

    def close(self):
        self._log_file.close()

Using TensorFlow backend.


### Plot Losses

In [0]:
# Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(25,8))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.savefig(STORAGE_LOCATION+'/VGG16#8_model_history.png')
    
    plt.show()

### Config

In [0]:
# configuration zone
BATCH_SIZE = 32
EPOCHS = 50
# here we assume the observations have been resized to 60x80
OBSERVATIONS_SHAPE = (None, 60, 80, 3)
ACTIONS_SHAPE = (None, 2)
SEED = 1234
# TODO: Give the path where you want to store the trained model
STORAGE_LOCATION = "/content/drive/"
# TODO: Give the path where the training data is stored
DATA_LOCATION = "/content/drive/"

### Load Data

In [0]:
# TODO: Specify name of the data
reader = Reader(DATA_LOCATION+'nameOfData.log')

observations, actions = reader.read()
actions = np.array(actions)
observations = np.array(observations)

x_train, x_validate, y_train, y_validate = train_test_split(observations, actions, test_size = 0.2, random_state = 2)

train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_datagen.fit(x_train)

validation_datagen = ImageDataGenerator()
validation_datagen.fit(x_validate)

### Model

In [0]:
base_model = VGG16(classes=2, input_shape=(60,80,3), weights=None, include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(2)(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Define the optimizer
optimizer = SGD(lr=0.01, momentum=0.001, nesterov=False)
# optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# Compile the model
model.compile(optimizer = optimizer ,
              loss = MSE, 
              metrics=["accuracy"])

es = EarlyStopping(monitor='val_loss', verbose=1, patience=30)
# TODO: Specify name of the model
mc = ModelCheckpoint(STORAGE_LOCATION+'nameOfModel.h5', monitor='val_loss', save_best_only=True)

history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=BATCH_SIZE),
                              validation_data = validation_datagen.flow(x_validate,y_validate, batch_size=BATCH_SIZE),
                              epochs = EPOCHS,
                              verbose=2,  # for hiding print statements
                              steps_per_epoch=observations.shape[0] // BATCH_SIZE,
                              callbacks=[es, mc],
                              shuffle=True)
plot_model_history(history)

### Evaluate Model

In [0]:
from keras.models import load_model
model = load_model(STORAGE_LOCATION+'nameOfModel.h5')
# Returns: [loss, accuracy]
model.evaluate(x_validate,y_validate)

In [0]:
num = 20
preds = model.predict(x_validate[:num])

for i in range(num):
  print("Pred: ", preds[i], "\tGT: ", y_validate[i])

Pred:  [0.9885629 1.0066956] 	GT:  [1. 1.]
Pred:  [0.7629906 0.9382076] 	GT:  [0.89646339 1.        ]
Pred:  [0.5505507 0.5559779] 	GT:  [0.58508306 0.57960364]
Pred:  [0.6189885  0.08426905] 	GT:  [0.59545575 0.10335627]
Pred:  [0.32440728 0.53234667] 	GT:  [0.4859214 0.6787653]
Pred:  [0.5814476  0.60452145] 	GT:  [0.55508671 0.60959999]
Pred:  [0.87181884 0.6696001 ] 	GT:  [0.72034147 0.44434523]
Pred:  [0.6246664  0.62242883] 	GT:  [0.58819145 0.57649525]
Pred:  [0.5550587  0.28357962] 	GT:  [0.50332463 0.19548739]
Pred:  [0.61963356 0.57528895] 	GT:  [0.59609417 0.56859253]
Pred:  [0.530713  0.5984092] 	GT:  [0.60016928 0.56451742]
Pred:  [0.12070434 0.58973235] 	GT:  [0.12537573 0.57343629]
Pred:  [0.9946303 0.9889532] 	GT:  [1. 1.]
Pred:  [0.49480438 0.22583602] 	GT:  [0.46940258 0.22940943]
Pred:  [0.37962127 0.695377  ] 	GT:  [0.1925249  0.50628712]
Pred:  [0.08598477 0.6732555 ] 	GT:  [0.03688926 0.66192276]
Pred:  [0.44178873 0.77202153] 	GT:  [0.41009757 0.75458913]
Pred:  